This script is to convert the configuration files in the ini format to json. Some modificatios to the key are required which can be done here while converting to json.

In [ ]:
import os
import json
import geopandas as gpd

def get_catchment_info(geopackage_file):
    # Read the GeoPackage file
    gdf = gpd.read_file(geopackage_file)
    
    # Create a dictionary mapping catchment IDs to catchment areas
    catchment_info = {row['id']: row['areasqkm'] for idx, row in gdf.iterrows()}
    return catchment_info

def get_catchment_id_from_filename(filename):
    # Extract the catchment ID from the filename (assuming the format "cat-<catchment_id>_config.ini")
    parts = filename.split('_')
    if len(parts) == 2 and parts[0].startswith('cat-'):
        catchment_id = parts[0].split('-')[1]
        return catchment_id
    else:
        return None

def ini_to_json(ini_file_path, json_file_path, catchment_info):
    data = {}
    
    # Extract catchment ID from the filename
    catchment_id = get_catchment_id_from_filename(os.path.basename(ini_file_path))
    catchment_id = 'cat-'+str(catchment_id)
    
    if catchment_id is not None:
        # Add catchment area information if catchment ID is available in the GeoPandas data
        catchment_area = catchment_info.get(catchment_id)
        if catchment_area is not None:
            data['catchment_area_km2'] = catchment_area
        else:
            print(f"Warning: No catchment area found for catchment ID {catchment_id}.")
    
    giuh_ordinates = []  # Initialize a list for giuh_ordinates
    nash_storage = []
    
    with open(ini_file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if '=' in line:
                keys, value = line.split('=', 1)
                keys = keys.split('.')
                value = value.split("[")[0]
                try:
                    value = float(value)
                except ValueError:
                    pass

                if keys[0] == 'surface_partitioning_scheme':
                    keys[0] = 'partition_scheme'

                if len(keys) == 1:
                    if keys[0] == 'nash_storage':
                        nash_storage = [float(x) for x in value.split(',')]
                    if keys[0] == 'giuh_ordinates':
                        giuh_ordinates = [float(x) for x in value.split(',')]
                    else:
                        data[keys[0]] = value
                else:
                    if keys[0] not in data:
                        data[keys[0]] = {}

                    if keys[0] == 'soil_params' and keys[1] == 'b':
                        keys[1] = 'bb'

                    data[keys[0]][keys[1]] = value
                    
                    if keys[0] == 'soil_params' and keys[1] == 'depth':
                        keys[1] = 'D' 

                    data[keys[0]][keys[1]] = value
    
    # Store giuh_ordinates as a list in the data dictionary
    data['giuh_ordinates'] = giuh_ordinates
    data['nash_storage'] = nash_storage
    # data['max_gw_storage'] *= 0.01
    # data['gw_storage'] = 0.01
    # data['soil_storage'] = 0.4
    # data['soil_params']['wltsmc'] = 0.01
    data['Cgw'] = 0.1
    # data['refkdt'] *= 0.01
    
    data['forcing_file'] = 'wb-2624577_retro_3_2022/forcings/'+catchment_id+'.csv'
    data['stand_alone'] = 1
    data['soil_scheme'] = "ode"
    
    # Write the data dictionary to the JSON file
    with open(json_file_path, 'w') as f:
        json.dump(data, f, indent=4)


def convert_all_ini_to_json(input_directory, output_directory, catchment_info):
    for filename in os.listdir(input_directory):
        if filename.endswith(".ini"):
            ini_file_path = os.path.join(input_directory, filename)
            json_file_path = os.path.join(output_directory, filename[:-11] + ".json")
            print(json_file_path)
            ini_to_json(ini_file_path, json_file_path, catchment_info)

# Specify the paths and file names
input_directory = 'wb-2624577_retro_3_2022/config'
output_directory = 'wb-2624577_retro_3_2022/json_input_config'
geopackage_file = 'wb-2624577_retro_3_2022/config/wb-2624577_upstream_subset.gpkg'

# Get catchment information from the GeoPackage file
catchment_info = get_catchment_info(geopackage_file)


# Use the function to convert INI files to JSON with catchment information
convert_all_ini_to_json(input_directory, output_directory, catchment_info)
